# Carregue as bibliotecas necessárias

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import json
import matplotlib.pyplot as plt
import numpy as np

from _utils import (NeuralNetwork, train_and_validate_with_kfold, train_and_validade,
plot_curvas_aprendizado, plot_pesos, test_model_with_new_data, 
previsao_without_normalization, previsao_with_normalization)

# Carregando o conjunto de dados
### Carregue o conjunto de dados gerado a partir do script dataset_generator.py e os separe em data e target

In [ ]:
dados = {}
with open("./dataset.json", "r") as arquivo:
    dados = json.load(arquivo)

data = [(d["coords"]) for d in dados["dados"]]
target = [d["params"] for d in dados["dados"]]

## Normalize os dados

In [ ]:
# Converta 'data' para um array bidimensional
data_flattened = [sample for series in data for sample in series]

# Crie uma instância do MinMaxScaler e ajuste aos dados
scaler = MinMaxScaler()
scaler.fit(data_flattened)

# Normalize os dados
data_normalized = [scaler.transform(series) for series in data]

scaler.fit(target)

# Normalize os rótulos
target_normalized = scaler.transform(target)

# Separando os dados em treino e teste
###  Separe os dados em 20% teste, 80% para treino e validação
#### Aqui temos dois tipos de dados: não normalizados e normalizados (com sufixo N)

In [ ]:
test_size = 0.2  # 20% dos dados para teste
train_dataN, test_dataN, train_targetN, test_targetN = train_test_split(
    data_normalized, target_normalized, test_size=test_size, random_state=42
)

train_dataN, val_dataN, train_targetN, val_targetN = train_test_split(
    train_dataN, train_targetN, test_size=test_size, random_state=42
)

train_data, test_data, train_target, test_target = train_test_split(
    data, target, test_size=test_size, random_state=42
)

train_data, val_data, train_target, val_target = train_test_split(
    train_data, train_target, test_size=test_size, random_state=42
)

# Definindo a arquitetura da rede neural
### Defina a arquitetura da rede neural a ser treinada

In [ ]:
num_layers = 1
units_fc = [32,]

modeloA, modeloB, modeloC = NeuralNetwork(num_layers, units_fc), NeuralNetwork(num_layers, units_fc), NeuralNetwork(num_layers, units_fc)

## Treinando o modelo
### Treino o modelo com os métodos:
#### - train_and_validate
#### - train_and_validate_with_kfold

In [ ]:
num_folds = 5
num_epochs = 25
batch_size = 32

modeloA, metricsA = train_and_validade(modeloA, train_data, train_target, val_data,
                                       val_target, num_epochs, batch_size)
modeloB, metricsB = train_and_validade(modeloB, train_dataN, train_targetN, val_dataN,
                                       val_targetN, num_epochs, batch_size)
modeloC, metricsC = train_and_validate_with_kfold(modeloC, train_dataN, train_targetN,
                                       num_epochs, num_folds, batch_size)

In [ ]:
# Salve o modelo
path = "modeloA.pth"
torch.save(modeloA.state_dict(), path)

path = "modeloB.pth"
torch.save(modeloB.state_dict(), path)

path = "modeloC.pth"
torch.save(modeloC.state_dict(), path)

# Curvas de aprendizado para treino e validação

In [ ]:
plot_curvas_aprendizado(metricsB, metricsC)

# Distribuição dos pesos

In [ ]:
plot_pesos(modeloC)

## Avalie o modelo com os dados de teste

In [ ]:
x_test_tensor = torch.Tensor(np.array(test_dataN))
y_test_tensor = torch.Tensor(np.array(test_targetN))

x_test_tensor1 = torch.Tensor(np.array(val_data))
y_test_tensor1 = torch.Tensor(np.array(val_target))

test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

test_dataset1 = TensorDataset(x_test_tensor1, y_test_tensor1)
test_loader1 = DataLoader(test_dataset1, batch_size=batch_size, shuffle=True)

test_lossesA = test_model_with_new_data(modeloA, test_loader1)
test_lossesB = test_model_with_new_data(modeloB, test_loader)
test_lossesC = test_model_with_new_data(modeloC, test_loader)

In [ ]:
# Plote a curva de perda durante os testes
plt.plot(test_lossesC, marker="o", linestyle="-")
plt.xlabel("Iteração do Teste")
plt.ylabel("Perda do Teste")
plt.title("Curva de Perda durante os Testes")
plt.grid(True)
plt.show()

# Teste novos dados

# Compare visualmente a previsão com o dado orignal

In [ ]:
# Função para gerar o gráfico
def plot_grafico(linhas, new_data, ax, titulo):
    # Defina o tamanho do gráfico
    x_min, x_max = 0, 6000
    y_min, y_max = 0, 8
    dmin, dmax = linhas[2], linhas[3]
    tl, th = linhas[0], linhas[1]

    x_line = [[x_min, dmin, dmin, dmax, dmax, x_max]]

    y_line = [[tl, tl, th, th, tl, tl]]

    ax.plot(x_line[0], y_line[0], color="red")

    ax.scatter([x[0] for x in new_data], [y[1] for y in new_data], c="blue")

    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)
    ax.set_title(titulo, loc="left")

In [ ]:
nDados = {}
with open("./newData.json", "r") as arquivo:
    nDados = json.load(arquivo)

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(6, 18))
previsaoA, pontos, params, mseA = previsao_without_normalization(modeloA, nDados)
previsaoB, pontos, params, mseB = previsao_with_normalization(modeloC, nDados, target, scaler)
# previsaoA, pontos, params, mseA = previsao_with_normalization(modeloC, nDados)
# # Converta a previsão para coordenadas no gráfico
# scaler.fit(target)
# previsaoA = scaler.inverse_transform(previsaoB.detach().numpy())
linhasA = previsaoA.tolist()[0]
scaler.fit(target)
previsao_c = scaler.inverse_transform(previsaoB.detach().numpy())
linhasB = previsao_c.tolist()[0]
plot_grafico(linhasB, pontos, axs[2], "C.   Modelo B")
axs[2].text(
    0.5,
    0.9,
    f"MSE: {mseB:.5f}",
    transform=axs[2].transAxes,
    ha="center",
    fontsize=14,
)
plot_grafico(linhasA, pontos, axs[1], "B.   Modelo A")
axs[1].text(
    0.5,
    0.9,
    f"MSE: {mseA:.5f}",
    transform=axs[1].transAxes,
    ha="center",
    fontsize=14,
)
plot_grafico(params[0], pontos, axs[0], "A. Rótulos Não Utilizados no Treinamento")
for ax in axs:
    ax.tick_params(axis="both", which="major", labelsize=14)
    ax.set_xlabel(ax.get_xlabel(), fontsize=14)
    ax.set_ylabel(ax.get_ylabel(), fontsize=14)
    ax.set_title(ax.get_title(), fontsize=14)
plt.savefig("comparação_modeloAB_original2203.png")
plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(18, 6))

# Converta a previsão para coordenadas no gráfico
scaler.fit(target)
previsao_c1 = scaler.inverse_transform(previsao1.detach().numpy())
linhas1 = previsao_c1.tolist()[0]
previsao_c2 = scaler.inverse_transform(previsao2.detach().numpy())
linhas2 = previsao_c2.tolist()[0]
previsao_c3 = scaler.inverse_transform(previsao3.detach().numpy())
linhas3 = previsao_c3.tolist()[0]

# Plote os gráficos
plot_grafico(linhas1, pontos, axs[0], "MSE")
plot_grafico(linhas2, pontos, axs[1], "R²")
plot_grafico(linhas3, pontos, axs[2], "Ponderada")

oParams = scaler.transform(params)
pondParams = previsao3.detach().numpy()
r2Params = previsao2.detach().numpy()
mseParams = previsao1.detach().numpy()

mse1 = calcula_mse(mseParams, oParams)
mse2 = calcula_mse(r2Params, oParams)
mse3 = calcula_mse(pondParams, oParams)

axs[0].text(
    0.5, 0.9, f"MSE: {mse1:.4f}", transform=axs[0].transAxes, ha="center", fontsize=14
)
axs[1].text(
    0.5, 0.9, f"MSE: {mse2:.4f}", transform=axs[1].transAxes, ha="center", fontsize=14
)
axs[2].text(
    0.5, 0.9, f"MSE: {mse3:.4f}", transform=axs[2].transAxes, ha="center", fontsize=14
)

for ax in axs:
    ax.tick_params(axis="both", which="major", labelsize=14)
    ax.set_xlabel(ax.get_xlabel(), fontsize=14)
    ax.set_ylabel(ax.get_ylabel(), fontsize=14)
    ax.set_title(ax.get_title(), fontsize=14)

plt.savefig("comparação_previsao_mse_r2_ponderada1.png")
plt.tight_layout()
plt.show()